Note: this notebook has been modified specifically for use on the oxford data set!

In [25]:
import csv
import pandas as pd
import numpy as np
import random
import csv

## 1. Preprocessing
### Create data

In [7]:
k_sev=1376
k_non_sev=4324
sev_p=[0.884,0.711,0.603,0.442,0.376,0.357,0.26,0.26,0.161,0.113,0.078,
       0.059,0.057,0.028]
non_sev_p=[0.814,0.657,0.442,0.057,0.28,0.128,0.131,0.109,0.121,0.135,0.097,
           0.057,0.058,0.051]
sev_n=[round(k_sev*i) for i in sev_p]
non_sev_n=[round(k_non_sev*i) for i in non_sev_p]

data = [sev_n, non_sev_n]
symptoms = ["Fever", "Cough", "Fatigue", "Dyspnea", "Sputum", "shortness", "Myalgia", "Chill", "Dizziness", "Headache", "sore", "Nausea", "Diarhea", "Congestion"]
covid_df = pd.DataFrame(data, columns = symptoms)

print(covid_df)

   Fever  Cough  Fatigue  Dyspnea  Sputum  shortness  Myalgia  Chill  \
0   1216    978      830      608     517        491      358    358   
1   3520   2841     1911      246    1211        553      566    471   

   Dizziness  Headache  sore  Nausea  Diarhea  Congestion  
0        222       155   107      81       78          39  
1        523       584   419     246      251         221  


Assume severe patients have a rating of around 2 

In [8]:
# multiply all values by scalar, with sd, around normal dist
def norm(center, sd):
    rating = np.abs(np.random.normal(center, sd)) 
    if rating > 10: rating = 10
    return rating

In [9]:
big_sev_list = []
big_non_sev_list = []

for symptom in symptoms:
    ones = [norm(8, 2) for i in range(covid_df[symptom][0])]
    zeroes = [0 for i in range(k_sev - covid_df[symptom][0])]
    new_col = ones + zeroes
    random.shuffle(new_col)
    big_sev_list.append(new_col)
    
for symptom in symptoms:
    ones = [norm(3,1) for i in range(covid_df[symptom][1])]
    zeroes = [0 for i in range(k_non_sev - covid_df[symptom][1])]
    new_col = ones + zeroes
    random.shuffle(new_col)
    big_non_sev_list.append(new_col)
    
big_sev_array = np.array(big_sev_list)
big_non_sev_array = np.array(big_non_sev_list)

big_sev_array = np.transpose(big_sev_array)
big_non_sev_array = np.transpose(big_non_sev_array)

print(big_sev_array.shape)
print(big_non_sev_array.shape)
big_data = np.concatenate((big_sev_array, big_non_sev_array), axis=0)
print(big_data.shape)
covid_df = pd.DataFrame(big_data, columns = symptoms)
print(covid_df.head)

#save final version

(1376, 14)
(4324, 14)
(5700, 14)
<bound method NDFrame.head of           Fever     Cough   Fatigue   Dyspnea     Sputum  shortness   Myalgia  \
0      9.594474  8.188026  5.355588  10.00000   0.000000   0.000000  9.627666   
1      6.466608  0.000000  0.000000   0.00000   0.000000   0.000000  0.000000   
2      5.871845  0.000000  0.000000   9.46079   0.000000   0.000000  7.327658   
3     10.000000  6.107289  7.805434   0.00000  10.000000   0.000000  0.000000   
4      0.000000  8.792409  0.000000   0.00000  10.000000   0.000000  0.000000   
...         ...       ...       ...       ...        ...        ...       ...   
5695   3.231399  0.000000  0.000000   0.00000   0.000000   0.000000  0.000000   
5696   2.641801  0.000000  0.000000   0.00000   3.173997   0.000000  0.000000   
5697   2.869739  0.000000  0.000000   0.00000   0.000000   0.000000  0.000000   
5698   2.110753  2.878624  0.000000   0.00000   3.737779   4.325959  1.805099   
5699   2.624982  3.654678  0.000000   0.00000 

In [27]:
covid_df.to_csv('data/generated-data.csv')

## 2. Graph Building
14 nodes

In [10]:
edges = []
for i in range(len(symptoms)):
        for j in range(i +1, len(symptoms)):
            edges.append((i,j))
triangles = []
for i in range(len(symptoms)):
        for j in range(i +1, len(symptoms)):
            for h in range(j + 1, len(symptoms)):
                triangles.append((i,j, h))
                
adj_matrix = np.zeros((len(symptoms), len(symptoms)))
curl = np.zeros((len(edges), len(triangles)))
neg_divergence = np.zeros((len(edges), len(symptoms)))
f = np.zeros((len(edges)))
W = np.zeros((len(edges), len(edges)))

In [19]:
# f, w
for index, row in covid_df.iterrows():
    for i, edge in enumerate(edges):
        W[i, i] += 1
        f[i] += (row[symptoms[edge[1]]] - row[symptoms[edge[0]]])
for i in range(len(edges)):
    f[i] = f[i]*1/W[i, i]

In [20]:
print(f)

print(W)

[-0.33208238 -0.68028908 -1.27404536 -1.08177495 -1.27610753 -1.36725259
 -1.38519242 -1.47120097 -1.50108826 -1.57348806 -1.63699613 -1.6386804
 -1.67084764 -0.3482067  -0.94196297 -0.74969256 -0.94402515 -1.03517021
 -1.05311003 -1.13911858 -1.16900588 -1.24140567 -1.30491375 -1.30659802
 -1.33876525 -0.59375627 -0.40148586 -0.59581845 -0.68696351 -0.70490333
 -0.79091188 -0.82079917 -0.89319897 -0.95670705 -0.95839132 -0.99055855
  0.19227041 -0.00206217 -0.09320724 -0.11114706 -0.19715561 -0.2270429
 -0.2994427  -0.36295077 -0.36463505 -0.39680228 -0.19433258 -0.28547764
 -0.30341747 -0.38942602 -0.41931331 -0.49171311 -0.55522118 -0.55690545
 -0.58907269 -0.09114506 -0.10908488 -0.19509343 -0.22498073 -0.29738053
 -0.3608886  -0.36257287 -0.39474011 -0.01793982 -0.10394837 -0.13383567
 -0.20623547 -0.26974354 -0.27142781 -0.30359504 -0.08600855 -0.11589584
 -0.18829564 -0.25180372 -0.25348799 -0.28565522 -0.02988729 -0.10228709
 -0.16579517 -0.16747944 -0.19964667 -0.0723998  -0.1

In [21]:
# adj_matrix
for index, row in covid_df.iterrows():
    for i in range(len(symptoms)):
        for j in range(i +1, len(symptoms)):
            adj_matrix[i, j] += (row[symptoms[i]] - row[symptoms[j]])/len(covid_df) #divide by #ppl with both symptoms
print(adj_matrix)

[[ 0.          0.66428131  1.36081691  2.54853783  2.16392953  2.5526629
   2.73498501  2.77087095  2.94291823  3.00270331  3.14752831  3.27456675
   3.27793588  3.34228164]
 [ 0.          0.          0.6965356   1.88425652  1.49964823  1.88838159
   2.0707037   2.10658964  2.27863692  2.338422    2.48324701  2.61028544
   2.61365457  2.67800033]
 [ 0.          0.          0.          1.18772092  0.80311262  1.19184599
   1.3741681   1.41005404  1.58210132  1.6418864   1.7867114   1.91374984
   1.91711897  1.98146473]
 [ 0.          0.          0.          0.         -0.38460829  0.00412507
   0.18644718  0.22233312  0.39438041  0.45416548  0.59899049  0.72602892
   0.72939806  0.79374381]
 [ 0.          0.          0.          0.          0.          0.38873336
   0.57105547  0.60694142  0.7789887   0.83877378  0.98359878  1.11063722
   1.11400635  1.1783521 ]
 [ 0.          0.          0.          0.          0.          0.
   0.18232211  0.21820805  0.39025534  0.45004041  0.5948654

In [22]:
# neg_divergence
for i in range(len(edges)):
    for j in range(len(symptoms)):
        if edges[i][0] == j:
            neg_divergence[i,j] = -1
        elif edges[i][1] == j:
            neg_divergence[i,j] = 1
print(neg_divergence)

[[-1.  1.  0. ...  0.  0.  0.]
 [-1.  0.  1. ...  0.  0.  0.]
 [-1.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... -1.  1.  0.]
 [ 0.  0.  0. ... -1.  0.  1.]
 [ 0.  0.  0. ...  0. -1.  1.]]


In [23]:
# curl
for j, tri in enumerate(triangles):
    for i, edge in enumerate(edges):
        if edge[0] in tri and edge[1] in tri:
            first_edge = tri.index(edge[0])
            if (first_edge + 1) % 3 == tri.index(edge[1]):
                curl[i, j] = 1
            else:
                curl[i,j] = -1
print(curl)

[[ 1.  1.  1. ...  0.  0.  0.]
 [-1.  0.  0. ...  0.  0.  0.]
 [ 0. -1.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  1.  0. -1.]
 [ 0.  0.  0. ...  0.  1.  1.]]


### Solving for r

In [24]:
right_side = np.matmul(np.transpose(neg_divergence), np.matmul(W, f))
left_side = np.matmul(np.matmul(np.transpose(neg_divergence), W), neg_divergence)
r = np.matmul(np.linalg.inv(left_side), right_side)
print(r)

[ 1.53125  1.28125  0.875    0.25     0.5      0.15625  0.125    0.21875
  0.03125  0.      -0.09375 -0.03125 -0.15625 -0.15625]


In [43]:
rank_df = pd.DataFrame({
    'symptoms': symptoms,
    'r': r
})
print(rank_df)

rank_df = rank_df.sort_values(by =['r'],  ascending = False)
rank_df = rank_df.reset_index(drop = True)
print(rank_df)

rank_df.to_csv('data/hodge_ranking.csv')

      symptoms        r
0        Fever  1.53125
1        Cough  1.28125
2      Fatigue  0.87500
3      Dyspnea  0.25000
4       Sputum  0.50000
5    shortness  0.15625
6      Myalgia  0.12500
7        Chill  0.21875
8    Dizziness  0.03125
9     Headache  0.00000
10        sore -0.09375
11      Nausea -0.03125
12     Diarhea -0.15625
13  Congestion -0.15625
      symptoms        r
0        Fever  1.53125
1        Cough  1.28125
2      Fatigue  0.87500
3       Sputum  0.50000
4      Dyspnea  0.25000
5        Chill  0.21875
6    shortness  0.15625
7      Myalgia  0.12500
8    Dizziness  0.03125
9     Headache  0.00000
10      Nausea -0.03125
11        sore -0.09375
12     Diarhea -0.15625
13  Congestion -0.15625


### Solving for c

In [74]:
right_side = np.matmul(np.transpose(curl), np.matmul(W, f))
left_side = np.matmul(np.matmul(np.transpose(curl), W), curl)
c = np.matmul(np.linalg.inv(left_side), right_side)
print(c)

[ 15728640. -13107200.  -7733248.   6733824.   2359296.   -983040.
  -1966080.  26214400.   6815744.  -8847360. -10485760.   3145728.
   1572864.  -1572864.   6160384.  -7602176.  12582912.   2228224.
   8912896.  -3014656. -12058624.   -393216.  14155776.   4587520.
  -5242880.   7667712. -11796480.  12582912.  -4194304.  -4194304.
  -4718592.   8486912. -18087936.  11272192.   3276800.  14155776.
  -4980736.   1015808.  -7864320.  -5242880.   7340032.   2064384.
   3145728.    786432.  -2883584.  -1081344.   1048576.   5931008.
 -14680064.   8912896. -12910592.   2916352. -12058624.    -65536.
   6553600.  -7864320.   7208960.   3276800.   7864320.    -65536.
  -1064960.  -5636096.   9961472.   5767168.  -1605632.   6160384.
 -10485760.  -2162688.  -9437184.    278528.  -1572864.   8912896.
   -983040.  -2031616. -24117248.   6553600.   8388608. -17301504.
  -4980736.   -499712.   5636096.   3145728.  -4063232.   1966080.]
